# Provenance with Python and Semantic Technologies

In this lab class, we look at a scientific workflow for interpreting aerosol observational data as measured by an observation system that is part of a larger [research infrastructure](https://ec.europa.eu/research/infrastructures/?pg=about) in order to detect the occurrence of [new particle formation events](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_01001359) on selected days in [Hyytiälä](http://sws.geonames.org/656888/), Finland. Detected events are then described, specifically their duration. Finally, event descriptions are used to create a dataset from which we compute average event duration.

While executing the workflow, we will generate numerous data files as well as [provenance](https://www.w3.org/TR/prov-o/) information. Provenance is recorded as [RDF](https://www.w3.org/RDF/) in files and we will look at them in detail as they are created. Finally, we will inspect the generated provenance using [SPARQL](https://www.w3.org/TR/sparql11-query/) queries.

Before we start, we need to load required Python modules as well as a few functions used in the workflow. Let's load first the required Python modules. Please execute the following (and all other) code blocks using ALT+ENTER or the `Run` button in the menu.

## Initialization

In [ ]:
import requests, io, pydot, json, glob, shutil, pathlib, pandas as pd, numpy as np
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import RDF, XSD
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer
from matplotlib import pyplot as plt
from urllib.parse import urlencode
from datetime import datetime, timedelta
from pytz import timezone
from rdflib import Graph, BNode
from rdflib.namespace import RDF
from rdflib.tools.rdf2dot import rdf2dot
from IPython.display import display, Image, Markdown
from prov.model import ProvDocument
from prov.dot import prov_to_dot
from shortid import ShortId
from sklearn.externals import joblib

The following code block creates a few sub folders needed to store data created in the workflow. If the folders exist we delete the content. Thus, if you like to start from scratch you can do so by executing the following code block.

In [ ]:
shutil.rmtree('average-duration', ignore_errors=True)
shutil.rmtree('event-dataset', ignore_errors=True)
shutil.rmtree('event-description', ignore_errors=True)
shutil.rmtree('observational-data', ignore_errors=True)
shutil.rmtree('provenance', ignore_errors=True)
pathlib.Path('average-duration').mkdir()
pathlib.Path('event-dataset').mkdir()
pathlib.Path('event-description').mkdir()
pathlib.Path('observational-data').mkdir()
pathlib.Path('provenance').mkdir()

Next we load a few functions used in the workflow. 

The first is called `fetch_data` and implements the functionality needed to fetch aerosol observational data for a given date (e.g., `2013-04-04`) as measured by an observation system located in [Hyytiälä](http://sws.geonames.org/656888/), Finland. The observation system is part of the Finnish [Station for Measuring Ecosystem-Atmosphere Relations](https://www.atm.helsinki.fi/SMEAR/) ([SMEAR](https://twitter.com/GlobalSMEAR)), an advanced research infrastructure with a history that goes back to the early '90. Specifically, the observation system is part of [SMEAR II](https://www.atm.helsinki.fi/SMEAR/index.php/smear-ii). Take a look at these links to get acquainted with SMEAR.

Naturally, the observational data are not fetched directly from the observation system. Rather, they are processed and published by the research infrastructure. Data publishing is supported by [SmartSMEAR](https://avaa.tdata.fi/web/avaa/-/smartsmear), a data visualization and download tool (see also [Junninen et al.](https://helda.helsinki.fi/bitstream/handle/10138/233466/ber14-4-447.pdf)). Conveniently, SmartSMEAR provides an [API](https://avaa.tdata.fi/web/smart/smear/api) through which we can programmatically access the required data. This is the task of the `fetch_data` function defined next.

In [ ]:
def fetch_data(date):
    time_from = timezone('Europe/Helsinki').localize(datetime.strptime(date, '%Y-%m-%d'))
    time_to = time_from + timedelta(days=1)

    query = {
        'table': 'HYY_DMPS', 'quality': 'ANY', 'averaging': 'NONE', 'type': 'NONE',
        'from': str(time_from), 'to': str(time_to), 'variables': 'd316e1,d355e1,d398e1,'\
        'd447e1,d501e1,d562e1,d631e1,d708e1,d794e1,d891e1,d100e2,d112e2,d126e2,d141e2,d158e2,'\
        'd178e2,d200e2,d224e2,d251e2,d282e2,d316e2,d355e2,d398e2,d447e2,d501e2,d562e2,d631e2,'\
        'd708e2,d794e2,d891e2,d100e3,d112e3,d126e3,d141e3,d158e3,d178e3,d200e3'
    }
    
    url = 'https://avaa.tdata.fi/smear-services/smeardata.jsp?' + urlencode(query)
    response = requests.post(url)

    return pd.read_csv(io.StringIO(response.text))

Researchers plot the daily aerosol observational data fetched from SmartSMEAR in order to determine whether an event occurred and to facilitate the description of detected events. The following function implements the plotting of observational data.

In [ ]:
def plot_data(data):
    d = data.copy(deep=True)
    d = d.iloc[:, 6:].values
    m = len(d)
    n = len(d[0])
    x = range(0, m)
    y = range(0, n)
    x, y = np.meshgrid(x, y)
    z = np.transpose(np.array([row[1:] for row in d]).astype(np.float))
    plt.figure(figsize=(10, 5), dpi=100)
    plt.pcolormesh(x, y, z)
    plt.plot((0, x.max()), (y.max()/2, y.max()/2), "r-")
    plt.colorbar()
    plt.xlim(xmax=m-1)
    x_ticks = np.arange(x.min(), x.max(), 6)
    x_labels = range(x_ticks.size)
    plt.xticks(x_ticks, x_labels)
    plt.xlabel('Hours')
    y_ticks = np.arange(y.min(), y.max(), 6)
    y_labels = ['3.16', '6.31', '12.6', '25.1', '50.1', '100']
    plt.yticks(y_ticks, y_labels)
    plt.ylabel('Diameter [nm]')
    plt.ylim(ymax=n-1)
    plt.show()

The following code block introduces several functions needed to create, write, and visualize provenance information. 

The most interesting function is `create_provenance`. It takes a set of `entities` from which an entity (`entity2`) is derived. A classical simple example is a dataset that is derived from another dataset. However, a dataset may also be derived from multiple datasets. The function also takes a reference to an activity as well as the start and end times of the activity.

There are three known activity types: the [data visualization activity](https://www.ebi.ac.uk/ols/ontologies/obi/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FOBI_0200111), the [averaging data transformation activity](https://www.ebi.ac.uk/ols/ontologies/obi/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FOBI_0200170), and the more general [data transformation activity](https://www.ebi.ac.uk/ols/ontologies/obi/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FOBI_0200000). These are all concepts of the [Ontology for Biomedical Investigations](http://obi-ontology.org/) (OBI). Follow the links provided here to learn more about these concepts. Don't be surprised by the term *Biomedical*. The domain here is earth and environmental sciences but these activities are ubiquitous in science. We can thus use ontologies originally developed within life sciences for our data-driven activities in earth sciences.

The function `create_provenance` uses this input and creates a provenance document. Specifically, it relates the entities, activity as well as the agent (more about this below) accordingly to the [Provenance Ontology](https://www.w3.org/TR/prov-o/) (PROV-O). [Take a look](https://www.w3.org/TR/prov-o/diagrams/starting-points.svg) at these three classes and relevant relations between them.

In [ ]:
def create_provenance(entities, entity2, activity, starttime, endtime):
    act = { # Defined activity types (classes)
        'obo:OBI_0200111': 'a data visualization activity',
        'obo:OBI_0200170': 'an averaging data transformation',
        'obo:OBI_0200000': 'a data transformation'
    }
    sid = ShortId()
    prov = ProvDocument()
    prov.add_namespace('file', 'file:')
    prov.add_namespace('orcid', 'https://orcid.org/')
    prov.add_namespace('obo', 'http://purl.obolibrary.org/obo/')
    prov.add_namespace('ex', 'http://example.org/')

    e2 = prov.entity('file:{}'.format(entity2))
    ag = prov.agent('orcid:{}'.format(orcid))
    ac = prov.activity('ex:activity_{}'.format(sid.generate()), starttime, endtime,
                       other_attributes={'prov:label': act[activity], 
                                         'prov:type': activity})
    
    prov.wasAttributedTo(e2, ag)
    prov.wasGeneratedBy(e2, ac)
    prov.wasAssociatedWith(ac, ag)
        
    for entity1 in entities:
        e1 = prov.entity('file:{}'.format(entity1))
        prov.wasDerivedFrom(e2, e1)
        prov.wasAttributedTo(e1, ag)
        prov.used(ac, e1)
    
    return prov

def write_provenance(provenance, file):
    provenance.serialize(file, format='rdf', rdf_format='ttl')
        
def visualize_provenance(provenance):
    dot = prov_to_dot(provenance)
    display(Image(dot.create_png()))
    
def new_provenance_file():
    return 'provenance/{}.ttl'.format(datetime.now().strftime('%Y-%m-%dT%H%M%S'))

The next code block implements the conversion of observational data into a feature vector classified by a trained artificial neural network (multilayer perceptron) to automatically assess whether an event occurred during the day. Although not central to the workflow here or the resulting provenance, such automated extraction of information about events is an interesting feature of advanced research infrastructure.

In [ ]:
classifier_detection = joblib.load('machine-learning-model/classifier-event-detection.pkl')
scaler_detection = joblib.load('machine-learning-model/scaler-event-detection.pkl')

def feature_vector(data):
    data_day_new = data.iloc[:, 6:20].values
    data_daytime_new = data.loc[(data['Hour'] >= 9) & (data['Hour'] <= 15)].iloc[:, 6:20].values
    data_nighttime_new = data.loc[(data['Hour'] < 9) | (data['Hour'] > 15)].iloc[:, 6:20].values
    ret = [
        np.sum(data_day_new),
        np.max(data_day_new),
        np.min(data_day_new),
        np.var(data_day_new),
        np.sum(data_nighttime_new),
        np.max(data_nighttime_new),
        np.min(data_nighttime_new),
        np.var(data_nighttime_new),
        np.sum(data_daytime_new) - np.sum(data_nighttime_new),
        np.max(data_daytime_new) - np.max(data_nighttime_new),
        np.min(data_daytime_new) - np.min(data_nighttime_new),
        np.var(data_daytime_new) - np.var(data_nighttime_new)
    ]
    
    return ret

Finally, we define a couple more functions that simply read and write data. Nothing suprising here.

In [ ]:
def read_data(file):
    return pd.read_csv(file)

def write_data(data, file, type='csv'):
    if type == 'csv':
        data.to_csv(file, index=False, encoding='utf-8')
    elif type == 'json':
        with open(file, 'w') as f:
            f.write(json.dumps(data))
    else:
        raise ValueError('Unsupported type: {}'.format(type))

## Configuration

Now that we have loaded modules and specialized functions as well as learned more about where the observational data are coming from and about auxiliary resources such as OBI and PROV-O, we move on to the actual workflow. Before we start, provide your [ORCID](https://orcid.org) iD. If you do not yet have an ORCID iD take a couple of minutes to [create one](https://orcid.org/register). If you do not want to create an ORCID iD you can provide your first and last name separated by a dash i.e., `FirstName-LastName`.

The ORCID iD is used here to identify the agent in provenance information. You can see this by closely inspecting the `create_provenance` function introduced earlier where the `orcid` variable is used to identify the agent. All provenance information created here is attributed to this agent.

In [ ]:
# Provide your ORCID iD or your FirstName-LastName
orcid = '0000-0001-5492-3212'

## Data Interpretation

Now we are all set up and ready to start executing our workflow. 

It begins with a *data interpretation* step. For a number of days, we fetch aerosol observational data from SmartSMEAR using the `fetch_data` function. The obtained data are first cached locally to a [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) file. This local caching is useful in case you want to visualize observational data for a day multiple times. Rather than fetching the data again from SmartSMEAR we can simply load them from the local cache. This pleases the SmartSMEAR admins as it reduces the number of requests. It makes also good sense here because the observational data do not change.

We provide several (example) days at which an event occurred. Please process some of the provided days. Your task is to record the beginning and end times of the event by looking at the visualization of observational data. Use the following code block to configure which day you want to analyse. When you have completed a day you will need to return to this code block and select another day.

Obviously, you can also select days other than the ones suggested here. You could choose a day that follows one provided here and see how the visualized observational data differs. To guide you, we also provide example days at which no event occurred.

In [ ]:
# Days to process
#
# Event days
# 2007-04-15, 2007-05-05, 2007-05-18, 2007-10-19, 2008-02-19, 2009-03-19, 2009-03-22 
# 2011-03-15, 2011-04-19, 2011-10-01, 2012-05-01, 2012-05-29, 2013-02-20, 2013-04-04
#
# Non Event days
# 2007-04-20, 2008-02-20, 2009-04-03, 2011-04-21, 2012-05-05, 2013-02-21

day = '2007-04-15'
observational_data_file = 'observational-data/{}.csv'.format(day)

Next we fetch and write observational data to a file. 

**You only need to do this once for a day.**

If you want to repeat the visualization and interpretation of a day for which you have already cached data you can skip the following code block and directly go to the next.

In [ ]:
write_data(fetch_data(day), observational_data_file)

Before you continue, make sure the corresponding file was written to your `observational-data` folder. Click on the file and take a look at the observational data obtained from SmartSMEAR. It is not easy to overview these data. Hence, researchers plot them to support their analysis, specifically detecting the occurrence of new particle formation events and the description of events.

Hence, we now visualize the data and based on the visualization describe the event and then store our description to a file. In addition to the day, the event description includes the beginning and end of the event i.e., the temporal duration. 

In addition to visualization, the following code block also uses a trained artificial neural network to automatically assess whether or not an event occurred on the day. Such automated assessment can support decision making. Naturally, automated assessments are not accurate to 100%. Hence, researchers need to review the machine assessment. To see the classifier in action, try some of the Non Event days provided above (e.g., `2007-04-20`).

We also record the `starttime` and `endtime` of workflow step execution. This is provenance information, specifically of the activity.

In [ ]:
starttime = datetime.now()

d = read_data(observational_data_file)
plot_data(d)

v = feature_vector(d)
v = np.array(v).reshape(1, -1)
v = scaler_detection.transform(v)

display(Markdown('<span style="color:red">Automated assessment by machine learning classifier: <span style="font-weight:bold">{} Day</span></span>'.format(classifier_detection.predict(v)[0])))

Now you can interpret the visualization. Your tasks is to determine at what time the event started and ended. A new particle formation event typically begins around noon with small particles (diameter around 3 nm, which is the detection limit of the instrument). The diameter range of interest to researchers is below 25 nm. The yellow color reflects high number of particles of a given diameter size.

Sometimes the event is clearly visible e.g., for `2013-04-04` but that's not always the case. Hence, there is a fairy amount of subjectivity that flows into this data interpretation process.

Try to determine the time at which events begin and end by looking at the visualization. If the yellow shape does not cover the entire diameter range from 3-25 nm try to (mentally) fit a curve to estimate the times. For the end time you can use the time when particle diameter is greater than 4-6 nm, or when the yellow shape flattens out, or when it exceeds 25 nm.

For the exercise here it is not relevant how accurately you estimate the beginning and end times. More important is that you get some variance in the recorded times. In the next code block you can record your estimated beginning and end time accordingly.

In [ ]:
event_description = {
    'beginning': '11:00',
    'end': '12:00'
}

Next we write the event description to a JSON file. In addition to the beginning and end times we also record the day. Finally, we stop recording the time used to perform the activity. Having executed the code block take a look at the JSON file in your `event-description` directory. 

In [ ]:
event_description['day'] = day

event_description_file = 'event-description/{}.json'.format(day)
write_data(event_description, event_description_file, type='json')

endtime = datetime.now()

Finally, we generate provenance information for the workflow step we just executed. The executed activity is [data visualization](https://www.ebi.ac.uk/ols/ontologies/obi/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FOBI_0200111) (obo:OBI_0200111). We write the provenance information to an RDF file (in [Turtle syntax](https://www.w3.org/TR/turtle/), `ttl`) to the `provenance` directory. Make a moment to look at the generated provenance. Finally, we visualize the provenance information as a graph. This visualization is a different representation of the same information you find in the RDF file.

In [ ]:
prov = create_provenance([observational_data_file], event_description_file, 'obo:OBI_0200111', starttime, endtime)
write_provenance(prov, new_provenance_file())
visualize_provenance(prov)

Before you proceed, take a close look at the visualization of provenance above. Read the graph and identify the entities, activity and agent. Which entity was derived from which other entity in the workflow step? How long did the activity take (note that the server time is [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time))?

Unless you have performed this workflow step for several days, you now need to go back up, select another day and perform the visual data interpretation for the selected day. Please iterate on this step for various days but make sure you move onto the next step early enough to have time to look at the next steps.

If you have completed visual data interpretation for several days the next step is to compute average event duration.

## Compute Average Event Duration

We have interpreted aerosol observational data for a number of days and we now want to compute the average duration of the described events. This is the next step in the workflow in which we generate new provenance information with further entities and different activity.

First, we create a dataset that includes the event descriptions as well as the duration. 

Take a look at the resulting dataset in the output. Are the computed durations correct?

The event dataset is written as a CSV file to your `event-dataset` folder.

In [ ]:
starttime = datetime.now()

entities = [] # File names of input entities tracked for provenance
event_dataset = []

for file in glob.glob('event-description/*.json'):
    entities.append(file)
    with open(file) as f:
        data = json.load(f)
        beginning = datetime.strptime('{} {}'.format(data['day'], data['beginning']), "%Y-%m-%d %H:%M")
        end = datetime.strptime('{} {}'.format(data['day'], data['end']), "%Y-%m-%d %H:%M")
        data['duration'] = (end - beginning)
        event_dataset.append(data)

df = pd.DataFrame(event_dataset, columns=['day', 'beginning', 'end', 'duration'])

event_dataset_file = 'event-dataset/{}.csv'.format(datetime.now().strftime('%Y-%m-%dT%H%M%S'))
write_data(df, event_dataset_file)

endtime = datetime.now()

df

Next we create provenance information for the performed workflow step. The executed activity is [data transformation](https://www.ebi.ac.uk/ols/ontologies/obi/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FOBI_0200000) (obo:OBI_0200000). Take a look at the generated provenance file in the `provenance` folder and a closer look at the generated visualization. Since numerous files are used as input, the visualization contains more entities. You can open the image in a separate browser tab by using the right mouse button.

In [ ]:
prov = create_provenance(entities, event_dataset_file, 'obo:OBI_0200000', starttime, endtime)
write_provenance(prov, new_provenance_file())
visualize_provenance(prov)

We now compute the mean duration and create an RDF description of the duration which we store to a file in your `average-duration` folder and print to the output. As before, take a look at the created file and output.

In [ ]:
starttime = datetime.now()

average = df['duration'].mean()

ns = 'http://www.w3.org/2006/time#'

node = BNode()

graph = Graph()
graph.bind('time', ns)

graph.add((node, RDF.type, URIRef('{}Duration'.format(ns))))
graph.add((node, URIRef('{}numericDuration'.format(ns)), Literal(average/timedelta(hours=1), datatype=XSD.decimal)))
graph.add((node, URIRef('{}unitType'.format(ns)), URIRef('{}unitHour'.format(ns))))

average_duration_file = 'average-duration/{}.ttl'.format(datetime.now().strftime('%Y-%m-%dT%H%M%S'))
graph.serialize(destination=average_duration_file, format='ttl')

endtime = datetime.now()

print(graph.serialize(format='ttl').decode('utf-8'))

Finally, we create provenance for this last step of the workflow. The executed activity is [averaging data transformation](https://www.ebi.ac.uk/ols/ontologies/obi/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FOBI_0200170) (obo:OBI_0200170). Take a look at the generated provenance file in the `provenance` folder and a closer look at the generated visualization.

In [ ]:
prov = create_provenance([event_dataset_file], average_duration_file, 'obo:OBI_0200170', starttime, endtime)
write_provenance(prov, new_provenance_file())
visualize_provenance(prov)

## Inspect Provenance

In this third part, we inspect provenance information using SPARQL queries. First, we read all RDF files with provenance information found in your `provenance` into a graph. Furthermore, we introduce an additional function which executes a SPARQL query and displays query results.

In [ ]:
g = Graph()

for file in glob.glob('provenance/*.ttl'):
    r = g.parse(file, format='turtle')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = io.BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(io.StringIO(output.getvalue().decode())))

The following query is for all entities that are derived from entities, and the corresponding activity. You can follow which event descriptions are derived from which observational data in data visualization activities; which event datasets are derived from which event descriptions in data transformation activities; and which average durations are derived from which event datasets in averaging data transformation activities. 

In [ ]:
query("""
PREFIX prov: <http://www.w3.org/ns/prov#> 
SELECT ?entity1 ?entity2 ?activity WHERE { 
  ?entity2 prov:wasDerivedFrom ?entity1 .
  ?entity2 prov:wasGeneratedBy [
      rdfs:label ?activity
  ]
}
ORDER BY (?entity1)
""")

The following query computes the average duration of activities, in seconds.

In [ ]:
query("""
PREFIX prov: <http://www.w3.org/ns/prov#> 
SELECT (avg(?duration) as ?duration) WHERE { 
  ?activity prov:startedAtTime ?startTime .
  ?activity prov:endedAtTime ?endTime .
#  ?activity rdfs:label "a data visualization activity"^^xsd:string
  BIND (
      (
          (3600*hours(?endTime)+60*minutes(?endTime)+seconds(?endTime)) - 
          (3600*hours(?startTime)+60*minutes(?startTime)+seconds(?startTime)
        )
    ) AS ?duration)
}
""")